In [1]:
import pandas as pd
import numpy as np
import time
import re
import pickle
import time

In [2]:
'''
Read in tracking data
'''

tracking2018 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2018.csv")
tracking2019 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2019.csv")
tracking2020 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2020.csv")

tracking2018.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right


In [3]:
#tracking2018.query("displayName == 'football'").iloc[50:110,]

(tracking2018['x'] + 270) % 360

0           311.32
1           311.68
2           312.05
3           312.43
4           312.84
             ...  
12777346     33.18
12777347     33.40
12777348     33.55
12777349     33.67
12777350     33.74
Name: x, Length: 12777351, dtype: float64

In [4]:
max_x = max(max(tracking2018.x),max(tracking2019.x),max(tracking2020.x))
x,s,a = (max(tracking2018.y),max(tracking2019.y),max(tracking2020.y))

print(f"Max x value = {max_x}")
print(f"Max y value = {(x,s,a)}")


Max x value = 148.76
Max y value = (88.89, 67.87, 67.59)


In [5]:


min_dir = min(tracking2020.dir)
max_dir = max(tracking2020.dir)
print(f"Min dir value = {min_dir}")
print(f"Max dir value = {max_dir}")

min_o = min(tracking2020.o)
max_o = max(tracking2020.o)
print(f"Min dir value = {min_o}")
print(f"Max dir value = {max_o}")

Min dir value = 0.0
Max dir value = 360.0
Min dir value = 0.0
Max dir value = 360.0


In [6]:
'''
Store all the desired plays
 - Was returned
 - No pass
 - No penalty yards
'''

play_data = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/plays.csv")
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull()", engine='python', inplace=False)

returned_plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
14,2018090600,2599,"(10:22) M.Bosher punts 46 yards to PHI 28, Cen...",3,4,16,ATL,Punt,Return,37267.0,...,NaN,NaN,NaN,3,6,NaN,46.0,9.0,37,36
26,2018090900,400,J.Tucker kicks 61 yards from BAL 35 to BUF 4. ...,1,0,0,BAL,Kickoff,Return,39470.0,...,NaN,NaN,NaN,7,0,NaN,61.0,28.0,33,45
27,2018090900,485,(7:25) (Punt formation) C.Bojorquez punts 46 y...,1,4,14,BUF,Punt,Return,46903.0,...,NaN,NaN,NaN,7,0,NaN,46.0,-1.0,47,82
28,2018090900,575,(6:37) (Punt formation) S.Koch punts 63 yards ...,1,4,9,BAL,Punt,Return,31018.0,...,NaN,NaN,NaN,7,0,NaN,63.0,4.0,59,36


In [7]:
'''
Ideas:
    Consider score, game clock, punt vs. kickoff, field/stadium (side advantages?)

'''
print("Columns:")
print(returned_plays.columns)
print("Sample")
print(returned_plays.iloc[:,10:])
print("Returner id")
print(returned_plays.loc[:,'returnerId'])

Columns:
Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'specialTeamsPlayType', 'specialTeamsResult',
       'kickerId', 'returnerId', 'kickBlockerId', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'penaltyCodes', 'penaltyJerseyNumbers',
       'penaltyYards', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult',
       'kickLength', 'kickReturnYardage', 'playResult',
       'absoluteYardlineNumber'],
      dtype='object')
Sample
      returnerId  kickBlockerId yardlineSide  yardlineNumber gameClock  \
3          44979            NaN          ATL              35  05:01:00   
14         29957            NaN          ATL              26  10:22:00   
26         42573            NaN          BAL              35  08:42:00   
27         46532            NaN          BUF              28  07:25:00   
28         37228            NaN          BAL              26  06:37:00   
...          ...            ...          ...           

In [8]:
'''
Function that calculates the (rounded) distance from a player
'''
def get_distances(x, player_pos):
    dist = round(np.linalg.norm(player_pos.values - x.values),3)
    return dist

In [9]:
'''
How we adjust x & y coordinates
 x : playDirection is way kicking team is running. Want x-coordinates defined by distance to the target endzone

'''

returned_plays.index[0]

3

In [10]:
### Ball carrier, opponents, teammates. Each group sorted by distance to ball carrier

pd.options.mode.chained_assignment = None

start_time = time.time()


# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    # if no returner id, don't include play
    if pd.isnull(returnerId):
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        

    #####################
    
    # shift x-axis based on target's endzone
    playDirection = play_df.loc[play_df.index[0],'playDirection']
    
    '''
    # Old x adjust
    if playDirection == 'left':
        play_df['adj_x'] = 100 - play_df['x']
    if playDirection == 'right':
        play_df['adj_x'] = play_df['x']
    '''
    
    if playDirection == 'left':
        play_df['adj_x'] = play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 270) % 360
        play_df['adj_dir'] = (play_df['dir'] + 270) % 360
    if playDirection == 'right':
        play_df['adj_x'] = 120 - play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 90) % 360
        play_df['adj_dir'] = (play_df['dir'] + 90) % 360
    
    play_df['adj_o'].round(2)
    play_df['adj_dir'].round(2)
    
    play_df['sin_adj_o'] = np.sin(play_df['adj_o'])
    play_df['cos_adj_o'] = np.cos(play_df['adj_o'])
    play_df['sin_adj_dir'] = np.sin(play_df['adj_dir'])
    play_df['cos_adj_dir'] = np.cos(play_df['adj_dir'])
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'next_adj_x'] = play_df.groupby('displayName').adj_x.shift(-1)
    play_df.loc[:, 'next_adj_y'] = play_df.groupby('displayName').adj_y.shift(-1)
    play_df.loc[:, 'next_adj_o'] = play_df.groupby('displayName').adj_o.shift(-1)
    play_df.loc[:, 'next_adj_dir'] = play_df.groupby('displayName').adj_dir.shift(-1)
    play_df.loc[:, 'next_sin_adj_o'] = play_df.groupby('displayName').sin_adj_o.shift(-1)
    play_df.loc[:, 'next_cos_adj_o'] = play_df.groupby('displayName').cos_adj_o.shift(-1)
    play_df.loc[:, 'next_sin_adj_dir'] = play_df.groupby('displayName').sin_adj_dir.shift(-1)
    play_df.loc[:, 'next_cos_adj_dir'] = play_df.groupby('displayName').cos_adj_dir.shift(-1)
    
    # remove NAs created by shifting; i.e. remove plays where there is no next play
    play_df.dropna(axis=0, how='any', subset=['x'], inplace=True)
    
    # create reward - defined as forward progress
    play_df['reward'] = round(play_df['next_x'] - play_df['x'], 3)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            other_team_name = 'home'
            if ball_carrier_team == 'home':
                other_team_name = 'away'

            '''
            # break into two teams
            ball_carrier_team = time_play_df.loc[time_play_df.team == ball_carrier_team, :]
            other_team = time_play_df.loc[time_play_df.team == other_team_name, :]
            '''
            
            # get distance from ball carrier
            ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
            time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['x','y']],))
        
            time_play_df['ball_carrier_bool'] = time_play_df.nflId != returnerId
            time_play_df['team_index'] = time_play_df.team == ball_carrier_team
            time_play_df.sort_values(by=["ball_carrier_bool","team_index","dist_from_ball_carrier"], inplace=True, ignore_index=True)
            
            #### 
            #Check to make sure ball carrier has football
            #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
            ####
            football_index = time_play_df.index[time_play_df.team == 'football']
            try:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
            except:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']

            time_play_df.loc[football_index,'o'] = 0
            time_play_df.loc[football_index,'dir'] = 0
            
            if (football_index != 1) | (football_distance > FOOTBALL_PLAYER_DIST):
                continue
                
            # save football state
            football_pos = time_play_df.loc[football_index,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y',]].to_numpy()
            # cut out from state
            idx = np.ones(len(time_play_df.index), dtype=bool)
            idx[football_index] = False
            time_play_df = time_play_df.iloc[idx] 

            states = time_play_df.loc[:,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']].to_numpy()

            rowDict['season'] = season
            rowDict['gameId'] = gameId
            rowDict['playId'] = playId
            
            rowDict["playIndex"] = index
            rowDict["timeIndex"] = time_index
            rowDict["time"] = time_string
            rowDict["football_pos"] = football_pos
            # add current state
            rowDict['state'] = states
            # add future states
            next_state = time_play_df.loc[:,['next_x','next_y', 'next_s', 'next_a', 'next_o', 'next_dir', 'next_adj_x', 'next_adj_y', 'next_adj_o', 'next_adj_dir', 'next_sin_adj_o', 'next_cos_adj_o', 'next_sin_adj_dir', 'next_cos_adj_dir']].to_numpy()
            rowDict['next_state'] = next_state
            # add reward
            try:
                rowDict['reward'] = time_play_df.loc[0,'reward']
            except:
                print("error getting reward")
                continue
                
            #print(states[0])
            #print('new state')
            #print(next_state[0])
            
            #print('action')
            #print(states[0]- next_state[0])
            #print((states[0]- next_state[0]).round(2))
                
            # can derive action by subtracting next position from current position
            action = states[0]- next_state[0]
            rowDict['action'] = action.round(2)
            
            
            # define features to calculate
            #rowDict['closest_opp_x'] = states[1,0]
            #rowDict['closest_opp_y'] = states[1,1]
            #rowDict['closest_abs_x'] = 
            #rowDict['closest_abs_y'] = 
            
            #third_state = time_play_df.loc[:,['third_x','third_y', 'third_s', 'third_a', 'third_o', 'third_dir']].to_numpy().reshape(-1)
            #next_action = next_state[0:2] - third_state[0:2]
            #rowDict['next_action'] = next_action.round(2)

            dict_list.append(rowDict)
            
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)

            
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
        
    
full_train_df = pd.DataFrame(dict_list)

full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

full_train_df = full_train_df.dropna(axis=0, how='any', inplace=False)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("from_csv_adj_angles.pkl")

print(F"total time: {total_time}  seconds")

Play 0 out of 4494
Play 250 out of 4494
Could not get returner id for index 362 at game 2018100701 for play 4281: <class 'Exception'>
Play 500 out of 4494
Could not get returner id for index 502 at game 2018101410 for play 2172: <class 'Exception'>
Play 750 out of 4494
Play 1000 out of 4494
Could not get returner id for index 1096 at game 2018120212 for play 1876: <class 'Exception'>
Play 1250 out of 4494
Could not get returner id for index 1470 at game 2018123011 for play 36: <class 'Exception'>
Play 1500 out of 4494
Play 1750 out of 4494
Play 2000 out of 4494
Play 2250 out of 4494
Could not get returner id for index 2443 at game 2019112408 for play 3869: <class 'Exception'>
Play 2500 out of 4494
Could not get returner id for index 2736 at game 2019121512 for play 4390: <class 'Exception'>
Play 2750 out of 4494
Play 3000 out of 4494
Play 3250 out of 4494
Play 3500 out of 4494
random error: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
r